# 2 БЛОК

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
#pip install category_encoders
import category_encoders as ce

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , RandomizedSearchCV,KFold,RepeatedKFold

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge , LogisticRegression , SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import svm
from sklearn.model_selection import learning_curve


from matplotlib.colors import ListedColormap
from sklearn.ensemble import RandomForestRegressor

In [ ]:
path_train = pd.read_csv('dim_train.csv') # содержит только имя файла, без имен папок !!!
df = pd.DataFrame(path_train)
display(df.head(),df.shape)

In [ ]:
df = df.drop(df[df['y'] == 0].index)
df = df.drop(df[df['x'] == 0].index)
df = df.drop(df[df['z'] == 0].index)
df.shape

In [ ]:
ax = sns.pairplot(df, hue = 'cut', palette= 'inferno')

In [ ]:
ax = sns.regplot(x = 'price' , y = 'y', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
ax = sns.regplot(x = 'price' , y = 'z', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
ax = sns.regplot(x = 'price' , y = 'depth', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
ax = sns.regplot(x = 'price' , y = 'table', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
df = df[df['x'] < 30 ]
df = df[df['y'] < 30 ]
df = df[(df['z'] < 30) & (df['z'] > 2)]
df = df[(df['depth'] < 75) & (df['depth'] > 50) ]
df = df[(df['table'] < 80) & (df['table'] > 45) ]
df.shape

In [ ]:
df = df.drop_duplicates(keep = 'last',ignore_index= True)

In [ ]:
corrmap = df_map.corr()
ax, f = plt.subplots(figsize = (12,9))
sns.heatmap(corrmap, annot= True)

In [ ]:
df_map = df.copy()
cat_feature = (df.dtypes == 'object')
cat_features = list(cat_feature[cat_feature].index)
le = LabelEncoder()

for cat in cat_features:
    df_map[cat] = le.fit_transform(df_map[cat])
display(df_map.head())

corrmap = df_map.corr()
ax, f = plt.subplots(figsize = (12,9))
sns.heatmap(corrmap, annot= True)


In [ ]:
cat_features = ['cut','color']
axis_features = ['x','y','z']
num_features = ['carat','depth','table']
y = np.array(df.price)
X = df.drop(columns=['price'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

In [ ]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(1,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map)),
    ('scaler', MinMaxScaler())
    ])
    
    
CT = ColumnTransformer(transformers=[("pol_std", axis_transformer, axis_features),
                                               ('num', StandardScaler(), num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])
display(CT)

X_train_CT = CT.fit_transform(X_train)
X_train_ = pd.DataFrame(X_train_CT)
X_test_ = pd.DataFrame(CT.transform(X_test)) # тестовый кусок трансформируем как траин
display(X_train_.head())

In [ ]:
def regmodel_param_test(
    alphas_to_try, X, y, cv, scoring = 'neg_mean_absolute_percentage_error', 
    model_name = 'LASSO', X_test = None, y_test = None, 
    draw_plot = False, filename = None):
    
    validation_scores = []
    train_scores = []
    results_list = []
    if X_test is not None:
        test_scores = []
        scorer = get_scorer(scoring)
    else:
        test_scores = None

    for curr_alpha in alphas_to_try:
        
        if model_name == 'LASSO':
            regmodel = Lasso(alpha = curr_alpha)
        elif model_name == 'Ridge':
            regmodel = Ridge(alpha = curr_alpha)
        else:
            return None

        results = cross_validate(
            regmodel, X, y, scoring=scoring, cv=cv, 
            return_train_score = True)

        validation_scores.append(np.mean(results['test_score']))
        train_scores.append(np.mean(results['train_score']))
        results_list.append(results)

        if X_test is not None:
            regmodel.fit(X,y)
            y_pred = regmodel.predict(X_test)
            test_scores.append(scorer(regmodel, X_test, y_test))
    
    chosen_alpha_id = np.argmax(validation_scores)
    chosen_alpha = alphas_to_try[chosen_alpha_id]
    max_validation_score = np.max(validation_scores)
    if X_test is not None:
        test_score_at_chosen_alpha = test_scores[chosen_alpha_id]
    else:
        test_score_at_chosen_alpha = None
        
    if draw_plot:
        regmodel_param_plot(
            validation_scores, train_scores, alphas_to_try, chosen_alpha, 
            scoring, model_name, test_scores, filename)
    
    return chosen_alpha, max_validation_score, test_score_at_chosen_alpha

In [ ]:
cv = RepeatedKFold(n_splits=15, n_repeats=3)
#(n_splits = 15, shuffle=True, random_state= 42)
lasso_alphas = np.linspace(1, 10, 10)

chosen_alpha, max_validation_score, test_score_at_chosen_alpha = regmodel_param_test(
        lasso_alphas, X_train_, y_train, 
        cv, scoring = 'neg_mean_absolute_percentage_error', model_name = 'LASSO', 
        X_test = X_test_, y_test = y_test, 
        draw_plot = True, filename = 'lasso_wide_search')

print("Chosen alpha: %.5f" % chosen_alpha)
print("Validation score: %.5f" % max_validation_score)
print("Test score at chosen alpha: %.5f" % test_score_at_chosen_alpha)

In [ ]:
def regmodel_param_plot(
    validation_score, train_score, alphas_to_try, chosen_alpha,
    scoring, model_name, test_score = None, filename = None):
    
    plt.figure(figsize = (8,8))
    sns.lineplot(y = validation_score, x = alphas_to_try, 
                 label = 'validation_data')
    sns.lineplot(y = train_score, x = alphas_to_try, 
                 label = 'training_data')
    plt.axvline(x=chosen_alpha, linestyle='--')
    if test_score is not None:
        sns.lineplot(y = test_score, x = alphas_to_try, 
                     label = 'test_data')
    plt.xlabel('alpha_parameter')
    plt.ylabel(scoring)
    plt.title(model_name + ' Regularisation')
    plt.legend()
    if filename is not None:
        plt.savefig(str(filename) + ".png")
    plt.show()

# KNN Regression 

In [ ]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}}]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map)),
    ('scaler', MinMaxScaler())
    ])
      
    
CT = ColumnTransformer(transformers=[ ("pol_std", axis_transformer, axis_features),
                                               ('num', StandardScaler(), num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])

pipe = Pipeline([('preprocessing', CT ), 
                 ('regressor',KNeighborsRegressor())])

cv = KFold(n_splits = 10, shuffle = True)

# Параметры для решетки(можно менять и смотреть)
p = np.arange(2,4)         
n_neighbors = [7,12,13]
weights = ['uniform','distance']
param_grid = [{
     'regressor__p':p,
     'regressor__n_neighbors': n_neighbors,
     'regressor__weights': weights}]

grid = GridSearchCV(pipe, param_grid, return_train_score = True,cv=cv, n_jobs = -1,scoring= 'r2')
grid.fit(X_train,y_train)
grid

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
table=pd.DataFrame(grid.cv_results_)
pv_table=table.pivot_table(index='param_regressor__p',columns='param_regressor__n_neighbors',values='mean_test_score')
pv_table
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(pv_table, annot=True, linewidths=.5, ax=ax)

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=42)


finish_pipe1 =  Pipeline([
            ('preprocessing', CT), 
            ('regressor',     KNeighborsRegressor(n_neighbors = 7, p = 2, weights = 'distance'))
            ])
finish_pipe1.fit(X_train, y_train)

print(finish_pipe1.score(X_train,y_train))
print(finish_pipe1.score(X_test,y_test))

In [ ]:
neighbors = np.arange(2, 15)
pp = np.arange(1,4)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
for i, k, in enumerate(neighbors):
    for p in (pp):
        knn = KNeighborsRegressor(n_neighbors = k, p = p)
        knn.fit(X_train_, y_train)
        train_accuracy[i] = knn.score(X_train_, y_train)
        test_accuracy[i] = knn.score(X_test_, y_test)

plt.title('k-NN: Varying Number of Neighbors')
plt.plot(neighbors, test_accuracy, label = 'Testing Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
for p in np.arange(1, 15):
    knk = KNeighborsRegressor(n_neighbors=7, p = p)
    strat_shuffle_split = KFold(n_splits=10)

cross_vall = cross_val_score(knk, X_train_, y_train, cv=strat_shuffle_split)

np.set_printoptions(precision=4)
print("________________")
print("p=",p)
print("Значения правильности перекрестной проверки:",cross_vall)
print("Среднее:",np.mean(cross_vall))
print("Средняя квадоатическая ощибка (MSE):",np.std(cross_vall))

# Машин опорн вектор

In [ ]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(1,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}}]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map)),
    ('scaler', MinMaxScaler())
    ])
      
    
CT = ColumnTransformer(transformers=[('axis', StandardScaler(), axis_features),
                                               ('num', StandardScaler(), num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])

pipe = Pipeline([('preprocessing',CT), 
                 ('classifier', svm.SVC(kernel='rbf'))])

C = np.array([1,10,100]) #10**3,10**4, 10**5,10**6,10**7
gamma = [0.1,1,5,10,100]

param_grid =[{'classifier__C': C,
     'classifier__gamma': gamma
     }]

grid = GridSearchCV(pipe, param_grid, cv = 5, return_train_score=True)
grid.fit(X_train,y_train)
table=pd.DataFrame(grid.cv_results_)
pv_table=table.pivot_table(index='param_classifier__C',columns='param_classifier__gamma',values='mean_test_score')
pv_table
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(pv_table, annot=True, linewidths=.5, ax=ax)

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))

# Lasso

In [ ]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}}]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map)),
    ('scaler', MinMaxScaler())
    ])

#("pol_std", axis_transformer, axis_features),
      
CT = ColumnTransformer(transformers=[('axis', StandardScaler(), axis_features),
                                               ('num', StandardScaler(), num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])


pipe = Pipeline([('preprocessing',CT), 
                 ('classifier', Lasso(max_iter = 300000, tol=1e-2))])

C = np.array([0.001, 0.01,0.1,1,10,100,10**3, 10**4]) 
param_grid = [{'classifier__alpha': C}]

grid = GridSearchCV(pipe, param_grid, cv = 10, return_train_score=True)
grid.fit(X_train,y_train)

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

# SGD Regressor

In [ ]:
CT = ColumnTransformer(transformers=[('axis', StandardScaler(), axis_features),
                                               ('num', StandardScaler(), num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])


#pipe = Pipeline([('preprocessing',CT), 
                 #('model', SGDRegressor())])
model = SGDRegressor(max_iter=3000000)

#n_alphas = 200
#alphas = np.logspace(-10, 0, n_alphas)
#param_grid = {'alpha': alphas,


param_grid = {'alpha': 10.0 ** -np.arange(1, 3),
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'learning_rate': ['constant', 'optimal', 'invscaling']
}

grid = RandomizedSearchCV(model, param_grid, cv = 10 , return_train_score=True)
grid.fit(X_train_,y_train)

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test_, y_test)))

# RandomForest regr


In [ ]:
#axis_transformer = Pipeline(steps=[
  #  ('polynom', PolynomialFeatures(1,include_bias=False))])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map))
    ])

#carat_transformer = Pipeline(steps=[
#    ('bining',KBinsDiscretizer(n_bins=7, encode='ordinal'))
#    ])
    
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[("pol_std", 'passthrough', axis_features),
                                               ('num', 'passthrough', num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])
display(CT)

X_CT = CT.fit_transform(X)
#X_train_ = pd.DataFrame(X_train_CT)
#X_test_ = pd.DataFrame(CT.transform(X_test)) # тестовый кусок трансформируем как траин
display(X_CT)

In [ ]:
# сначала выполняем код с СТ и потом уже здесь выполняем, тк изначально трансформ данные


# Параметры для решетки(можно менять и смотреть)
n_estimators = [5,10,45,50,70,75,80,85,90,100,200,300,400,500]
max_features= [2,7,8,9,10,11,15,18,20,25,30]
min_samples_leaf = [1,2,5,10,15]
estimator = RandomForestRegressor()

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf}



grid = RandomizedSearchCV(estimator, param_grid, return_train_score = True, cv = 5 , n_jobs = -1, scoring= 'neg_mean_absolute_percentage_error')
grid.fit(X_CT,y)
grid

In [ ]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

In [ ]:
print("----------------- Обучили и тестировали -------------------")
print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_CT, y)))

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

finish_pipe_RFR =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestRegressor(n_estimators = 200, min_samples_leaf = 1 , max_features = 25))
            ])

finish_pipe_RFR.fit(X, y)

print(finish_pipe_RFR.score(X,y))

#print(finish_pipe_RFR.score(X_test,y_test))

In [ ]:
estimator = RandomForestRegressor(n_estimators = 70, min_samples_leaf= 1 , max_features = 10)
estimator.fit(X_train_,y_train)
display(estimator.score(X_train_, y_train))
estimator.score(X_test_, y_test)

In [ ]:
sc_train1 =[]
sc_test1 = []
sc_train2 =[]
sc_test2 = []
n_estimators = [10,25, 40, 50, 60, 70, 100, 200]
for n in n_estimators:
    rf = RandomForestRegressor(n_estimators= n,max_features= 10, min_samples_leaf = 1,random_state=0, n_jobs = -1)
    rf.fit(X_train_, y_train)
    sc_train_n = rf.score(X_train_, y_train)
    sc_test_n = rf.score(X_test_, y_test)
    sc_train1.append(sc_train_n)
    sc_test1.append(sc_test_n)  
    
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(121)
#ax2 = fig.add_subplot(122)
ax1.plot(n_estimators,sc_train1)
ax1.plot(n_estimators,sc_test1)
ax1.set_ylim(0.95,1)


In [ ]:

train_sizes, train_scores, test_scores = learning_curve(RandomForestRegressor(n_estimators = 100, min_samples_leaf= 1 , max_features = 10), X_train_, y_train, cv=5, n_jobs=-1)


train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.subplots(1, figsize=(10,10))
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# это код для того, чтобы сделать бины на данных - каратах
# применяем его и потом делаем СТ(либо в блоке с СТ есть бин трансформер)
#сильно не влияет на обучение модели, поэтому не применял
kb = KBinsDiscretizer(n_bins = 4, encode='ordinal', strategy='uniform') # добавили strategy='uniform', по умолчанию была квантильная
df['carat'] = kb.fit_transform(pd.DataFrame(df.carat))

plt.subplots(1, figsize=(15,10))
ax = sns.scatterplot(x = 'carat' , y = 'price', data = df)

# БЛОК 3

In [ ]:
path_train = pd.read_csv('dim_train.csv') 
df = pd.DataFrame(path_train)
display(df.head(),df.shape)

path_test = pd.read_csv('dim_test.csv')
df1 = pd.DataFrame(path_test)
display(df1.head(),df1.shape)

In [ ]:
df = df.drop(df[df['y'] == 0].index)
df = df.drop(df[df['x'] == 0].index)
df = df.drop(df[df['z'] == 0].index)
df.shape

In [ ]:
df = df[df['x'] < 30 ]
df = df[df['y'] < 30 ]
df = df[(df['z'] < 30) & (df['z'] > 2)]
df = df[(df['depth'] < 75) & (df['depth'] > 50) ]
df = df[(df['table'] < 80) & (df['table'] > 45) ]
df.shape

In [ ]:
cat_features = ['cut','color']
axis_features = ['x','y','z']
num_features = ['carat','depth','table']
y = np.array(df.price)
X = df.drop(columns=['price'])

In [ ]:
clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':7, 'VVS1':6, 'VVS2':5, 'VS1':4, 'VS2':3, 
               'SI1':2, 'SI2':1, 'I1':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = clarity_map))
    ])

# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[("axis", 'passthrough', axis_features),
                                               ('num', 'passthrough', num_features),
                                               ('cat', OneHotEncoder(), cat_features),
                                               ("ordinal_map", clarity_transformer, ['clarity'])])
display(CT)

#X_train_CT = CT.fit_transform(X_train)
#X_train_ = pd.DataFrame(X_train_CT)
#X_test = pd.DataFrame(CT.transform(X_test)) # тестовый кусок трансформируем как траин
#display(X_train_)

In [ ]:
# есть 0 значения => убираем их
df1.describe()

In [ ]:
# тут есть 2 нулевых значения
df1 = df1.drop(df1[df1['y'] == 0].index)
df1 = df1.drop(df1[df1['x'] == 0].index)
df1 = df1.drop(df1[df1['z'] == 0].index)
df1.shape

In [ ]:
# делаем маску
mask = df1.index
#pd.DataFrame(mask)

In [ ]:
# посмотрим на данные, нужно ли будет убирать выбросы, как делали с df
ax = sns.pairplot(df1, hue = 'cut', palette= 'inferno')
# таких значительных выбросов нет, поэтому оставляем так

In [ ]:
# Предсказание и обучение

In [ ]:
finish_pipe_RFR =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestRegressor(n_estimators = 85, min_samples_leaf = 2 , max_features = 15))
            ])

finish_pipe_RFR.fit(X, y)
print(finish_pipe_RFR.score(X,y))

In [ ]:
y_predict = finish_pipe_RFR.predict(df1)
y_predict.shape